In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

print("Data Integration: - Crashes & Person Datasets")
print(f"\nIntegration Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Data Integration: - Crashes & Person Datasets

Integration Start Time: 2025-11-21 13:49:28


In [4]:
# Load cleaned datasets
print("\n LOADING CLEANED DATASETS")
print("-" * 80)

# Load cleaned crashes data
#crashes_cleaned = pd.read_csv('data/crashes_cleaned.csv', low_memory=False)
crashes_cleaned = pd.read_csv('data/Motor_Vehicle_Collisions_Crashes.csv', low_memory=False)
print(f"✓ Crashes (cleaned): {crashes_cleaned.shape}")
print(f"  Columns: {crashes_cleaned.columns.tolist()}")

# Load cleaned person data
#person_cleaned = pd.read_csv('data/person_cleaned.csv', low_memory=False)
person_cleaned = pd.read_csv('data/Motor_Vehicle_Collisions_Person.csv.csv', low_memory=False)
print(f"\n✓ Person (cleaned): {person_cleaned.shape}")
print(f"  Columns: {person_cleaned.columns.tolist()}")

# Display basic info
print(f"\n DATASET OVERVIEW:")
print(f"  Crashes: {len(crashes_cleaned):,} collision records")
print(f"  Person: {len(person_cleaned):,} person records (occupants, pedestrians, cyclists)")


 LOADING CLEANED DATASETS
--------------------------------------------------------------------------------


NameError: name 'crashes_cleaned' is not defined

In [ ]:
# 1- Pre-integration analysis: Check COLLISION_ID coverage
print("\n PRE-INTEGRATION ANALYSIS: COLLISION_ID COVERAGE")
print("=" * 80)

# Check for missing COLLISION_ID in both datasets
crashes_missing_id = crashes_cleaned['COLLISION_ID'].isna().sum()
person_missing_id = person_cleaned['COLLISION_ID'].isna().sum()

print(f"\n1. Missing COLLISION_ID:")
print(f"   • Crashes dataset: {crashes_missing_id:,} ({crashes_missing_id / len(crashes_cleaned) * 100:.2f}%)")
print(f"   • Person dataset: {person_missing_id:,} ({person_missing_id / len(person_cleaned) * 100:.2f}%)")

# Check unique COLLISION_IDs
crashes_unique_ids = crashes_cleaned['COLLISION_ID'].nunique()
person_unique_ids = person_cleaned['COLLISION_ID'].nunique()

print(f"\n2. Unique COLLISION_IDs:")
print(f"   • Crashes dataset: {crashes_unique_ids:,}")
print(f"   • Person dataset: {person_unique_ids:,}")

# Find common and unique IDs
crashes_ids = set(crashes_cleaned['COLLISION_ID'].dropna())
person_ids = set(person_cleaned['COLLISION_ID'].dropna())

common_ids = crashes_ids.intersection(person_ids)
crashes_only = crashes_ids - person_ids
person_only = person_ids - crashes_ids

print(f"\n3. COLLISION_ID Overlap:")
print(f"   • Common IDs (in both datasets): {len(common_ids):,}")
print(f"   • IDs only in Crashes: {len(crashes_only):,}")
print(f"   • IDs only in Person: {len(person_only):,}")
print(f"   • Match rate: {len(common_ids) / len(crashes_ids) * 100:.2f}%")

# Analyze person records per collision
print(f"\n4. Person Records per Collision:")
persons_per_collision = person_cleaned.groupby('COLLISION_ID').size()
print(f"   • Average: {persons_per_collision.mean():.2f} persons/collision")
print(f"   • Median: {persons_per_collision.median():.0f} persons/collision")
print(f"   • Max: {persons_per_collision.max():.0f} persons/collision")
print(f"   • Distribution:")
distribution = persons_per_collision.value_counts().sort_index().head(10)
for count, freq in distribution.items():
    print(f"     - {count} person(s): {freq:,} collisions ({freq / len(persons_per_collision) * 100:.1f}%)")

In [ ]:
# 2- Decision point: Choose integration strategy
print("\n INTEGRATION STRATEGY SELECTION")
print("=" * 80)

print("""
STRATEGY OPTIONS:
1. LEFT JOIN (Crashes ← Person): Keep all crashes, add person details
2. INNER JOIN (Crashes ↔ Person): Keep only collisions with person data
3. ONE-TO-MANY JOIN: Each crash can have multiple person records

CHOSEN STRATEGY: LEFT JOIN (Option 1)
JUSTIFICATION:
  ✓ Preserves all collision records from crashes dataset
  ✓ Maintains complete crash history even without person details
  ✓ Allows analysis of crashes with/without reported injuries
  ✓ Handles one-to-many relationship (1 crash → multiple persons)

EXPECTED OUTCOME:
  • Result will have MORE rows than crashes dataset
  • Each crash with N persons → N rows in final dataset
  • Crashes without person data → 1 row with NaN person fields
""")

In [ ]:
# 3- Perform the integration
print("\n PERFORMING DATA INTEGRATION")
print("=" * 80)

# Check if COLLISION_ID exists and has correct data type
print("Pre-merge data type check:")
print(f"  Crashes COLLISION_ID dtype: {crashes_cleaned['COLLISION_ID'].dtype}")
print(f"  Person COLLISION_ID dtype: {person_cleaned['COLLISION_ID'].dtype}")

# Ensure COLLISION_ID is same type (convert to int64 where possible)
crashes_cleaned['COLLISION_ID'] = pd.to_numeric(crashes_cleaned['COLLISION_ID'], errors='coerce').astype('Int64')
person_cleaned['COLLISION_ID'] = pd.to_numeric(person_cleaned['COLLISION_ID'], errors='coerce').astype('Int64')

# Perform LEFT JOIN
print("\nExecuting LEFT JOIN...")
start_time = pd.Timestamp.now()

integrated_data = crashes_cleaned.merge(
    person_cleaned,
    on='COLLISION_ID',
    how='left',
    suffixes=('_CRASH', '_PERSON'),
    indicator=True
)

end_time = pd.Timestamp.now()
merge_duration = (end_time - start_time).total_seconds()

print(f"✓ Join completed in {merge_duration:.2f} seconds")
print(f"\n INTEGRATION RESULTS:")
print(f"   • Original crashes: {len(crashes_cleaned):,} rows")
print(f"   • Original persons: {len(person_cleaned):,} rows")
print(f"   • Integrated dataset: {len(integrated_data):,} rows")
print(f"   • Expansion factor: {len(integrated_data) / len(crashes_cleaned):.2f}x")

# Analyze merge indicator
merge_stats = integrated_data['_merge'].value_counts()
print(f"\n MERGE STATISTICS:")
for merge_type, count in merge_stats.items():
    percentage = count / len(integrated_data) * 100
    if merge_type == 'both':
        print(f"   • Matched (crash + person data): {count:,} ({percentage:.1f}%)")
    elif merge_type == 'left_only':
        print(f"   • Crash only (no person data): {count:,} ({percentage:.1f}%)")
    elif merge_type == 'right_only':
        print(f"   • Person only (no crash data): {count:,} ({percentage:.1f}%)")

# Drop the merge indicator column
integrated_data = integrated_data.drop('_merge', axis=1)

In [ ]:
# 4- Post-integration data quality assessment
print("\n POST-INTEGRATION DATA QUALITY ASSESSMENT")
print("=" * 80)

# Check for duplicate columns (from suffixes)
duplicate_cols = [col for col in integrated_data.columns if '_CRASH' in col or '_PERSON' in col]
if len(duplicate_cols) > 0:
    print(f"\n DUPLICATE COLUMNS DETECTED (from merge suffixes):")
    for col in sorted(duplicate_cols):
        print(f"   • {col}")
    print(f"\n   Action required: Resolve {len(duplicate_cols)} duplicate columns")
else:
    print("\n✓ No duplicate columns detected")

# Check for new missing values
print(f"\n MISSING VALUES IN PERSON COLUMNS:")
person_cols = [col for col in integrated_data.columns if col in person_cleaned.columns and col != 'COLLISION_ID']
missing_summary = []
for col in person_cols[:10]:  # Show first 10 person columns
    missing_count = integrated_data[col].isna().sum()
    missing_pct = missing_count / len(integrated_data) * 100
    missing_summary.append({
        'Column': col,
        'Missing': missing_count,
        'Percentage': f"{missing_pct:.1f}%"
    })

missing_df = pd.DataFrame(missing_summary)
print(missing_df.to_string(index=False))

# Data type consistency check
print(f"\n DATA TYPE CONSISTENCY:")
print(f"   • Total columns: {len(integrated_data.columns)}")
print(f"   • Numeric columns: {len(integrated_data.select_dtypes(include=[np.number]).columns)}")
print(f"   • Object/String columns: {len(integrated_data.select_dtypes(include=['object']).columns)}")
print(f"   • Datetime columns: {len(integrated_data.select_dtypes(include=['datetime64']).columns)}")

In [ ]:
# 5- Handle CRASH_DATETIME conflict (if both datasets have datetime columns)
print("\n RESOLVING DATETIME COLUMN CONFLICTS")
print("=" * 80)

# Check if we have duplicate datetime columns
datetime_cols = [col for col in integrated_data.columns if 'DATETIME' in col.upper() or 'CRASH_DATE' in col.upper()]

if len(datetime_cols) > 1:
    print(f"Found {len(datetime_cols)} datetime-related columns:")
    for col in datetime_cols:
        print(f"   • {col}: {integrated_data[col].dtype}")

    # Strategy: Keep CRASH_DATETIME from crashes dataset (more authoritative)
    if 'CRASH_DATETIME_CRASH' in integrated_data.columns:
        integrated_data['CRASH_DATETIME'] = integrated_data['CRASH_DATETIME_CRASH']
        integrated_data = integrated_data.drop(['CRASH_DATETIME_CRASH', 'CRASH_DATETIME_PERSON'], axis=1,
                                               errors='ignore')
        print(f"\n✓ Resolved: Kept CRASH_DATETIME from crashes dataset")
    elif 'CRASH_DATETIME' in integrated_data.columns and integrated_data['CRASH_DATETIME'].notna().sum() > 0:
        print(f"\n✓ No conflict: Single CRASH_DATETIME column already present")
else:
    print("✓ No datetime column conflicts detected")

In [ ]:
# 6- Analyze integrated data relationships
print("\n ANALYZING CRASH-PERSON RELATIONSHIPS")
print("=" * 80)

# Crashes with person data vs without
crashes_with_persons = integrated_data[integrated_data['PERSON_TYPE'].notna()].groupby('COLLISION_ID').size()
crashes_without_persons = len(crashes_cleaned) - len(crashes_with_persons)

print(f"1. Crashes by person data availability:")
print(
    f"   • With person data: {len(crashes_with_persons):,} ({len(crashes_with_persons) / len(crashes_cleaned) * 100:.1f}%)")
print(
    f"   • Without person data: {crashes_without_persons:,} ({crashes_without_persons / len(crashes_cleaned) * 100:.1f}%)")

# Person types distribution in integrated data
if 'PERSON_TYPE' in integrated_data.columns:
    print(f"\n2. Person types in integrated dataset:")
    person_types = integrated_data['PERSON_TYPE'].value_counts(dropna=False)
    for ptype, count in person_types.items():
        pct = count / len(integrated_data) * 100
        print(f"   • {ptype}: {count:,} ({pct:.1f}%)")

# Injury analysis
if 'PERSON_INJURY' in integrated_data.columns:
    print(f"\n3. Injury distribution:")
    injuries = integrated_data['PERSON_INJURY'].value_counts(dropna=False)
    for injury, count in injuries.head(5).items():
        pct = count / len(integrated_data) * 100
        print(f"   • {injury}: {count:,} ({pct:.1f}%)")

# Cross-validation: Compare injury counts
if all(col in integrated_data.columns for col in ['NUMBER OF PERSONS INJURED', 'PERSON_INJURY']):
    print(f"\n4. Cross-validation: Injury count consistency")

    # Count injured persons per collision in integrated data
    injured_per_collision = integrated_data[integrated_data['PERSON_INJURY'] == 'Injured'].groupby(
        'COLLISION_ID').size()

    # Get crashes that have both metrics
    sample_crashes = integrated_data[integrated_data['NUMBER OF PERSONS INJURED'].notna()].drop_duplicates(
        'COLLISION_ID').head(5)

    print(f"   Sample comparison (first 5 crashes):")
    for idx, row in sample_crashes.iterrows():
        collision_id = row['COLLISION_ID']
        reported = row['NUMBER OF PERSONS INJURED']
        actual = injured_per_collision.get(collision_id, 0)
        match = "✓" if reported == actual else "⚠"
        print(f"   {match} Collision {collision_id}: Reported={reported}, Actual={actual}")

In [ ]:
# 7- Save integrated dataset
print("\n SAVING INTEGRATED DATASET")
print("=" * 80)

output_path = 'data/integrated_crashes_person.csv'
integrated_data.to_csv(output_path, index=False)

print(f"✓ Integrated dataset saved to: {output_path}")
print(f"  • Rows: {len(integrated_data):,}")
print(f"  • Columns: {len(integrated_data.columns)}")
print(f"  • File size: {pd.read_csv(output_path).memory_usage(deep=True).sum() / 1024 ** 2:.1f} MB")

# Create summary statistics
summary = {
    'Integration Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Crashes (original)': len(crashes_cleaned),
    'Persons (original)': len(person_cleaned),
    'Integrated Rows': len(integrated_data),
    'Expansion Factor': f"{len(integrated_data) / len(crashes_cleaned):.2f}x",
    'Crashes with Persons': len(crashes_with_persons),
    'Crashes without Persons': crashes_without_persons,
    'Match Rate': f"{len(crashes_with_persons) / len(crashes_cleaned) * 100:.1f}%"
}

summary_df = pd.DataFrame([summary]).T
summary_df.columns = ['Value']
print(f"\n INTEGRATION SUMMARY:")
print(summary_df.to_string())

print("\n✅ DATA INTEGRATION COMPLETE!")